# Script to plot trajectory data from CBWaves

1) Open and read all trajectory related variables 

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

# Open the HDF5 file
file_path = '/Users/jaykalinani/Doc/PhD_Work/17_BBH/BBH_trajectories/old/CircularOrbit_a9_Lflip_q1by7_sep20.h5'  # Replace with your HDF5 file path
with h5py.File(file_path, 'r') as hdf:
    # Print the structure of the HDF5 file
    print("Keys: %s" % hdf.keys())
    t = hdf['t'][:]  

    #BH1
    x1 = hdf['x1'][:] 
    y1 = hdf['y1'][:] 
    z1 = hdf['z1'][:] 
    a1x = hdf['a1x'][:]
    a1y = hdf['a1y'][:]
    a1z = hdf['a1z'][:]
    vx1 = hdf['vx1'][:] 
    vy1 = hdf['vy1'][:] 
    vz1 = hdf['vz1'][:]
    
    #BH2
    x2 = hdf['x2'][:] 
    y2 = hdf['y2'][:]
    z2 = hdf['z2'][:] 
    a2x = hdf['a2x'][:]
    a2y = hdf['a2y'][:]
    a2z = hdf['a2z'][:]
    vx2 = hdf['vx2'][:] 
    vy2 = hdf['vy2'][:] 
    vz2 = hdf['vz2'][:]


2) Plotting BH1 coordinates

In [ ]:
#Plotting BH1 coordinates
plt.plot(t, x1, label='x1')
plt.plot(t, y1, label='y1')
plt.plot(t, z1, label='z1')
print(x1[0], y1[0], z1[0])
plt.xlabel('t')
plt.ylabel('d')
plt.legend()
plt.title('BH1 coordinates')
plt.show()

3) Plotting BH2 coordinates

In [ ]:
#Plotting BH2 coordinates
plt.plot(t, x2, label='x2')
plt.plot(t, y2, label='y2')
plt.plot(t, z2, label='z2')
print(x2[0], y2[0], z2[0])
plt.xlabel('t')
plt.ylabel('d')
plt.legend()
plt.title('BH2 coordinates')
plt.show()

4) Plotting BH1 spin

In [ ]:
#Plotting BH1 spin
plt.plot(t, a1x, label='a1x')
plt.plot(t, a1y, label='a1y')
plt.plot(t, a1z, linestyle=':', label='a1z')
plt.xlabel('t')
plt.ylabel('a')
plt.legend()
plt.title('BH1 spin components')
plt.show()

5) Plotting BH2 spin

In [ ]:
#Plotting BH2 spin
plt.plot(t, a2x, label='a2x')
plt.plot(t, a2y, label='a2y')
plt.plot(t, a2z, linestyle=':', label='a2z')
plt.xlabel('t')
plt.ylabel('a')
plt.legend()
plt.title('BH2 spin components')
plt.show()

6) Plotting BH1 velocities

In [ ]:
#Plotting BH1 vel
plt.plot(t, vx1, label='vx1')
plt.plot(t, vy1, label='vy1')
plt.plot(t, vz1, label='vz1')
plt.xlabel('t')
plt.ylabel('v')
plt.legend()
plt.title('BH1 velocities')
plt.show()

7) Plotting BH2 velocities

In [ ]:
#Plotting BH2 vel
plt.plot(t, vx2, label='vx2')
plt.plot(t, vy2, label='vy2')
plt.plot(t, vz2, label='vz1')
plt.xlabel('t')
plt.ylabel('v')
plt.legend()
plt.title('BH1 velocities')
plt.show()

8) Movie for BH trajectories with jet cones

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D, art3d
from matplotlib.animation import FuncAnimation

def plot_cone(ax, base, direction, height, radius, color='cyan', alpha=0.1):
    """Plots a 3D cone on the given axis."""
    u = np.linspace(0, 2 * np.pi, 20)
    v = np.linspace(0, 1, 10)
    
    x = radius * np.outer(np.cos(u), v)
    y = radius * np.outer(np.sin(u), v)
    z = height * np.outer(np.ones(np.size(u)), v)
    
    # Normalize direction vector
    direction = np.array(direction)
    
 #   if np.linalg.norm(direction) == 0:
 #       # If direction is zero, use a default direction (e.g., the z-axis or some other perpendicular direction)
 #       direction = np.array([0, 0, 1])  # Use the z-axis as a default if the direction is zero
 #   direction = np.array(direction, dtype=float)
 #   direction /= np.linalg.norm(direction)
    
    # Compute the rotation matrix for the direction
    z_axis = np.array([0, 0, 1])  # We want to rotate to align with the z-axis
    rot_axis = np.cross(z_axis, direction)
    rot_angle = np.arccos(np.dot(z_axis, direction))
    
    if np.linalg.norm(rot_axis) > 1e-6:
        rot_axis /= np.linalg.norm(rot_axis)
        
        cos_a = np.cos(rot_angle)
        sin_a = np.sin(rot_angle)
        one_c = 1 - cos_a
        ux, uy, uz = rot_axis
        
        R = np.array([
            [cos_a + ux**2 * one_c, ux * uy * one_c - uz * sin_a, ux * uz * one_c + uy * sin_a],
            [uy * ux * one_c + uz * sin_a, cos_a + uy**2 * one_c, uy * uz * one_c - ux * sin_a],
            [uz * ux * one_c - uy * sin_a, uz * uy * one_c + ux * sin_a, cos_a + uz**2 * one_c]
        ])
        
        # Apply rotation to cone
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                xyz = np.dot(R, np.array([x[i, j], y[i, j], z[i, j]]))
                x[i, j], y[i, j], z[i, j] = xyz
    
    # Translate the cone to the base position
    x += base[0]
    y += base[1]
    z += base[2]
    
    cone = ax.plot_surface(x, y, z, color=color, alpha=alpha)
    return cone

def plot_flipped_cone_bh2(ax, base, height, radius, color='red', alpha=0.1):
    """Plots the flipped bottom cone for BH2 with zero spin (negative z-direction)."""
    # Direction for flipped cone (negative z-direction for zero spin case)
    direction = [0, 0, -1]

    # Create the cone mesh
    u = np.linspace(0, 2 * np.pi, 20)
    v = np.linspace(0, 1, 10)
    
    x = radius * np.outer(np.cos(u), v)
    y = radius * np.outer(np.sin(u), v)
    z = height * np.outer(np.ones(np.size(u)), v)
    
    # Rotate the cone to make sure it points in the correct direction
    # Flip the z-axis values (invert direction for negative z)
    z = -z

    # Translate the cone to the base position
    x += base[0]
    y += base[1]
    z += base[2]
    
    # Plot the surface (flipped cone)
    return ax.plot_surface(x, y, z, color=color, alpha=alpha)


# Load data
file_path = '/Users/jaykalinani/Doc/PhD_Work/17_BBH/BBH_trajectories/data/CircularOrbit_a9_Lflip_q1by7_sep20.h5'
with h5py.File(file_path, 'r') as hdf:
    t = hdf['t'][:]
    x1, y1, z1 = hdf['x1'][:], hdf['y1'][:], hdf['z1'][:]
    a1x, a1y, a1z = hdf['a1x'][:], hdf['a1y'][:], hdf['a1z'][:]
    x2, y2, z2 = hdf['x2'][:], hdf['y2'][:], hdf['z2'][:]
    a2x, a2y, a2z = hdf['a2x'][:], hdf['a2y'][:], hdf['a2z'][:]

# Parameters
radii_bh1, radii_bh2 = 7.0, 1.0
num_frames = 2400
cone_height = 25
cone_radius = cone_height * np.tan(np.radians(15))
indices = np.linspace(0, len(t) - 1, num_frames).astype(int)
t = t[indices]
x1, y1, z1, a1x, a1y, a1z = x1[indices], y1[indices], z1[indices], a1x[indices], a1y[indices], a1z[indices]
x2, y2, z2, a2x, a2y, a2z = x2[indices], y2[indices], z2[indices], a2x[indices], a2y[indices], a2z[indices]

# Setup the figure and 3D axis
fig = plt.figure(figsize=(10, 8))  # Increase the figure size for better resolution
ax = fig.add_subplot(111, projection='3d')

# Set up the plot limits for zooming in on the black holes
zoom_factor = 0.7  # Adjust to zoom in more or less
x_min, x_max = np.min([x1, x2]), np.max([x1, x2])
y_min, y_max = np.min([y1, y2]), np.max([y1, y2])
z_min, z_max = np.min([z1, z2]), np.max([z1, z2])

ax.set_xlim([x_min / zoom_factor, x_max / zoom_factor])
ax.set_ylim([y_min / zoom_factor, y_max / zoom_factor])
ax.set_zlim([z_min / zoom_factor, z_max / zoom_factor])

bh1, = ax.plot([x1[0]], [y1[0]], [z1[0]], 'o', markersize=radii_bh1 * 4, color='black')
bh2, = ax.plot([x2[0]], [y2[0]], [z2[0]], 'o', markersize=radii_bh2 * 4, color='black')
traj1, = ax.plot([], [], [], lw=1, color='purple', alpha=0.3)
traj2, = ax.plot([], [], [], lw=1, color='orange', alpha=0.3)
time_text = ax.text2D(0.05, 0.95, '', transform=ax.transAxes, fontsize=12, verticalalignment='top')

# Initialize the spin vectors
spin1 = None
spin2 = None

cones = []

def update(num):
    global cones
    global spin1, spin2  # Declare spin1 and spin2 as global to modify them inside the function
    while cones:
        cones.pop().remove()
    
    bh1.set_data([x1[num]], [y1[num]])
    bh1.set_3d_properties([z1[num]])
    bh2.set_data([x2[num]], [y2[num]])
    bh2.set_3d_properties([z2[num]])

    # Update the spin vectors
    if spin1 is not None:
        spin1.remove()
    if spin2 is not None:
        spin2.remove()
    
    spin1 = ax.quiver(x1[num], y1[num], z1[num], a1x[num], a1y[num], a1z[num], color='blue', length=6, normalize=True)
    spin2 = ax.quiver(x2[num], y2[num], z2[num], a2x[num], a2y[num], a2z[num], color='orange', length=6, normalize=True)
    
    start_idx = max(0, num - 50)
    traj1.set_data(x1[start_idx:num+1], y1[start_idx:num+1])
    traj1.set_3d_properties(z1[start_idx:num+1])
    traj2.set_data(x2[start_idx:num+1], y2[start_idx:num+1])
    traj2.set_3d_properties(z2[start_idx:num+1])
    

    direction1 = np.array([a1x[num], a1y[num], a1z[num]], dtype=float)
    if np.linalg.norm(direction1) > 0:
        direction1 /= np.linalg.norm(direction1)  # Normalize only if nonzero

    # For BH2, calculate the velocity in the xy-plane and find the perpendicular direction
    velocity2 = [x2[num] - x2[num - 1], y2[num] - y2[num - 1], z2[num] - z2[num - 1]]
    velocity2_xy = np.array(velocity2)[:2]  # Only use the xy components
    velocity2_xy /= np.linalg.norm(velocity2_xy)  # Normalize it

    # Now, compute the perpendicular direction using cross-product in the xy-plane
    # In 2D, a 90-degree rotation can be achieved by swapping the components and negating one
    perpendicular_direction2 = np.array([velocity2_xy[1], -velocity2_xy[0]])#np.array([-velocity2_xy[1], velocity2_xy[0]])

    # Ensure the perpendicular direction points in the correct direction
    # To check this, we need to compare the direction to the expected perpendicular motion

    # Ensure it's a unit vector
    perpendicular_direction2 /= np.linalg.norm(perpendicular_direction2)

    # For the direction of the cone, we add the z-component as 0 (no vertical component in orbital motion)
    direction2 = np.array([perpendicular_direction2[0], perpendicular_direction2[1], 0])
    
    
    # For BH2, spin is zero, so use a fixed direction for the flipped bottom cone
    direction2 = [0, 0, -1]  # Zero spin means flipped direction
    
    # Plot the top cone for BH1
    cones.append(plot_cone(ax, [x1[num], y1[num], z1[num]], direction1, cone_height, cone_radius, 'blue'))
    cones.append(plot_cone(ax, [x1[num], y1[num], z1[num]], -direction1, cone_height, cone_radius, 'blue'))

    # Plot the top cone for BH2 (non-flipped, direction is fixed)
    cones.append(plot_cone(ax, [x2[num], y2[num], z2[num]], direction2, cone_height, cone_radius, 'red'))
    # Plot the bottom cone for BH2 (flipped direction)
    #cones.append(plot_cone(ax, [x2[num], y2[num], z2[num]], -direction2, cone_height, cone_radius, 'red'))
    cones.append(plot_flipped_cone_bh2(ax, [x2[num], y2[num], z2[num]], cone_height, cone_radius, 'red'))
    
    time_text.set_text(f't = {t[num]:.1f} M')

    if num == num_frames - 1:
        # Remove individual cones
        while cones:
            cones.pop().remove()
    
        # Compute final spin direction (assuming BH1's spin represents the remnant spin)
        final_spin_direction = np.array([a1x[-1], a1y[-1], a1z[-1]], dtype=float)
        if np.linalg.norm(final_spin_direction) > 0:
            final_spin_direction /= np.linalg.norm(final_spin_direction)  # Normalize

         # Use the final merged BH location (approximate as the center of mass)
        final_bh_position = [(x1[-1] + x2[-1]) / 2, (y1[-1] + y2[-1]) / 2, (z1[-1] + z2[-1]) / 2]

         # Plot a single orange cone for the remnant BH
        cones.append(plot_cone(ax, final_bh_position, final_spin_direction, cone_height, cone_radius, 'orange'))
        cones.append(plot_cone(ax, final_bh_position, -final_spin_direction, cone_height, cone_radius, 'orange'))    

    return bh1, bh2, traj1, traj2, time_text

ani = FuncAnimation(fig, update, frames=num_frames, interval=20, blit=False)
# Add labels and a legend
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
# ax.legend()  # Uncomment if you want to include a legend

# Adjust layout to remove extra white space and padding
plt.tight_layout(pad=0)
ani.save('bbh_simulation_with_cones.mp4', writer='ffmpeg', fps=24, dpi=300)
plt.show()


9) Movie for BH Hill spheres

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation

# Path to the ASCII file
file_path = "/Users/jaykalinani/ET/simulations/Frontera/CBD_prod_MPWZ9_691_140_280_rmin_26_rmax_25e3_q_1by7_d_30/output-0000/trajectories.asc"

# Load data from the file
data = np.loadtxt(file_path)

# Extract position and velocity columns
xi1x, xi1y = data[:, 2], data[:, 3]  # BH1 positions
xi2x, xi2y = data[:, 5], data[:, 6]  # BH2 positions
v1x, v1y = data[:, 8], data[:, 9]  # BH1 velocities
v2x, v2y = data[:, 11], data[:, 12]  # BH2 velocities

# Black hole masses and radii
m1, m2 = 0.875, 0.125
r1, r2 = 2 * m1, 2 * m2  # Radii of the black holes
M = m1 + m2  # Total mass

# Create a figure and axis
fig, ax = plt.subplots(figsize=(9, 9))
ax.set_xlim(-40, 40)
ax.set_ylim(-40, 40)
ax.set_xlabel("x-coordinate", fontsize=14)
ax.set_ylabel("y-coordinate", fontsize=14)
ax.set_title("L-flip CBD: BH trajectories", fontsize=16)
ax.grid(which="both", linestyle="--", linewidth=0.5, alpha=0.7)
ax.minorticks_on()

# Add trajectories (lines) with blue and green colors, and alpha=0.7
ax.plot(xi1x, xi1y, color="tab:blue", alpha=0.7, label="BH1 Trajectory")
ax.plot(xi2x, xi2y, color="tab:green", alpha=0.7, label="BH2 Trajectory")

# Initialize black holes and Hill spheres as circles
bh1 = Circle((xi1x[0], xi1y[0]), r1, color="black")
bh2 = Circle((xi2x[0], xi2y[0]), r2, color="black")
hill_sphere1 = Circle((xi1x[0], xi1y[0]), 0, color="tab:blue", fill=False, linestyle="--")
hill_sphere2 = Circle((xi2x[0], xi2y[0]), 0, color="tab:green", fill=False, linestyle="--")

# Add patches to the axis
ax.add_patch(bh1)
ax.add_patch(bh2)
ax.add_patch(hill_sphere1)
ax.add_patch(hill_sphere2)
ax.legend(fontsize=12)

# Update function for animation
def update(frame):
    # Update black hole positions
    bh1.set_center((xi1x[frame], xi1y[frame]))
    bh2.set_center((xi2x[frame], xi2y[frame]))

    # Update Hill spheres
    separation = np.sqrt((xi1x[frame] - xi2x[frame])**2 + (xi1y[frame] - xi2y[frame])**2)
    r_H1 = separation * (m1 / (3 * M))**(1/3)
    r_H2 = separation * (m2 / (3 * M))**(1/3)
    hill_sphere1.set_radius(r_H1)
    hill_sphere1.set_center((xi1x[frame], xi1y[frame]))
    hill_sphere2.set_radius(r_H2)
    hill_sphere2.set_center((xi2x[frame], xi2y[frame]))

    return bh1, bh2, hill_sphere1, hill_sphere2

# Create the animation
num_frames = len(xi1x)
anim = FuncAnimation(fig, update, frames=num_frames, interval=50, blit=False)

# Save the animation as an MP4 file
output_path = "BH_trajectories_with_Hill_spheres_animation.mp4"
anim.save(output_path, writer="ffmpeg", dpi=600)
print(f"Animation saved as {output_path}")

# Show the plot
plt.show()


10) Computing total number of orbits

In [ ]:
import numpy as np
from scipy.integrate import quad

# Constants (set in natural units G = c = 1)
G = 1.0
c = 1.0

# Parameters
q = 1 / 7  # Mass ratio
m2 = 1 / (1 + q)
m1 = q * m2
M = m1 + m2  # Total mass
mu = m1 * m2 / M  # Reduced mass
a0 = 30  # Initial separation

# Keplerian orbital period
def orbital_period(a):
    return 2 * np.pi * np.sqrt(a**3 / (G * M))

# Rate of change of separation (da/dt)
def da_dt(a):
    return -64 / 5 * G**3 * mu * M**2 / (c**5 * a**3)

# Number of orbits integrand: dN/da = 1 / T(a) * (dt/da)
def dN_da(a):
    T_a = orbital_period(a)
    dt_da = 1 / da_dt(a)
    return 1 / T_a * dt_da

# Integrate to find total number of orbits
a_values = np.linspace(a0, 0.01, 1000)  # Avoid division by zero
N_orbits, _ = quad(dN_da, 0.01, a0)  # Lower limit set to a small value

print(f"Total number of orbits: {N_orbits:.2f}")
